# Obtención de datos utilizando la API de Spotify

La API de Spotify es una herramienta poderosa que nos acceder a una amplia gama de datos relacionados con la música disponible en la plataforma de Spotify. A través de esta API, es posible obtener información detallada sobre canciones, álbumes, artistas, listas de reproducción...

In [16]:
import pandas as pd
import spotipy
from spotipy.oauth2 import SpotifyClientCredentials
from dotenv import load_dotenv
import os

In [12]:
# Ocultamos nuestras credenciales y las definimos
load_dotenv()

client_id = os.getenv('CLIENT_ID')
client_secret = os.getenv('CLIENT_SECRET')

In [13]:
auth_manager = SpotifyClientCredentials(client_id=client_id, client_secret=client_secret)
sp = spotipy.Spotify(auth_manager=auth_manager)

In [14]:
def buscar_listas_reproduccion(query, limit=10):
    # Buscar listas de reproducción con el término dado
    resultados = sp.search(q=query, type='playlist', limit=limit)
    playlists = resultados['playlists']['items']
    
    # Crear una lista para almacenar información de las playlists
    playlist_data = []
    
    for playlist in playlists:
        playlist_info = {
            'nombre_playlist': playlist['name'],
            'id_playlist': playlist['id'],
            'total_tracks': playlist['tracks']['total']
        }
        playlist_data.append(playlist_info)
    
    return playlist_data

# Buscar listas con temas de anuncios de televisión
query = "anuncios TV"
playlists = buscar_listas_reproduccion(query, limit=10)


In [15]:
def obtener_detalles_de_tracks(playlists):
    tracks_data = []
    
    for playlist in playlists:
        tracks = sp.playlist_tracks(playlist['id_playlist'])['items']
        for item in tracks:
            track = item['track']
            
            # Verificar si el track es None
            if track is None:
                continue
            
            # Obtener características adicionales de audio, si están disponibles
            audio_features = sp.audio_features(track['id'])[0] if track['id'] else None
            
            # Verificar que audio_features no sea None antes de acceder a sus valores
            if audio_features is None:
                audio_features = {}
            
            track_info = {
                'nombre_cancion': track.get('name', ''),  # Utiliza una cadena vacía si el nombre es None
                'duracion_ms': track.get('duration_ms', 0),  # Utiliza 0 si no se encuentra la duración
                'popularidad': track.get('popularity', 0),  # Utiliza 0 si no hay popularidad
                'acousticness': audio_features.get('acousticness', 0.0),  # Utiliza un valor por defecto si no hay audio features
                'danceability': audio_features.get('danceability', 0.0),
                'energy': audio_features.get('energy', 0.0),
                'instrumentalness': audio_features.get('instrumentalness', 0.0),
                'liveness': audio_features.get('liveness', 0.0),
                'loudness': audio_features.get('loudness', 0.0),
                'speechiness': audio_features.get('speechiness', 0.0),
                'tempo': audio_features.get('tempo', 0.0),
                'valence': audio_features.get('valence', 0.0),
                'key': audio_features.get('key', 0),
                'mode': audio_features.get('mode', 0),
                'time_signature': audio_features.get('time_signature', 4)
            }
            tracks_data.append(track_info)
    
    return pd.DataFrame(tracks_data)

# Crear el DataFrame con todos los detalles de los tracks
df_tracks_detalles = obtener_detalles_de_tracks(playlists)

# Mostrar las primeras filas del DataFrame
df_tracks_detalles


Max Retries reached


SpotifyException: http status: 429, code:-1 - /v1/audio-features/?ids=3QnHWkNMY2mpy494Bis0ly:
 Max Retries, reason: too many 429 error responses